In [ ]:
!pip install tensorflow
!pip install keras
!pip install tensorflow_datasets

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
import tensorflow_datasets as tfds
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau



In [ ]:
(train_ds, test_ds) = tfds.load('cats_vs_dogs', split=["train[:80%]", "train[80%:]"], as_supervised=True)


In [ ]:
IMG_SIZE = 128
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [ ]:
train_ds = train_ds.map(preprocess).batch(32)
test_ds = test_ds.map(preprocess).batch(32)

In [ ]:
## Model
model = Sequential()
model.add(Conv2D(32, (3,3), activation="relu", input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D())
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D())
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(64, activation="relu", kernel_regularizer=l2))
model.add(Dropout(0.3))
model.add(Dense(32, activation="relu", kernel_regularizer=l2))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
reduce_lr = ReduceLROnPlateau(patience=5, min_delta=0.5)
early_stop = EarlyStopping(patience=8, restore_best_weights=True)
history = model.fit(train_ds,
                    epochs=20,
                    validation_data=test_ds,
                    verbose=True,
                    callbacks=[reduce_lr, early_stop])

Epoch 1/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 39s 55ms/step - accuracy: 0.5546 - loss: 1.8330 - val_accuracy: 0.6047 - val_loss: 0.8949 - learning_rate: 0.0010
Epoch 2/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 0.7149 - loss: 0.8004 - val_accuracy: 0.6427 - val_loss: 0.7884 - learning_rate: 0.0010
Epoch 3/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.7683 - loss: 0.6416 - val_accuracy: 0.7825 - val_loss: 0.5990 - learning_rate: 0.0010
Epoch 4/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.8007 - loss: 0.5723 - val_accuracy: 0.8048 - val_loss: 0.5486 - learning_rate: 0.0010
Epoch 5/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 27s 45ms/step - accuracy: 0.8210 - loss: 0.5235 - val_accuracy: 0.8181 - val_loss: 0.5300 - learning_rate: 0.0010
Epoch 6/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 0.8361 - loss: 0.5103 - val_accuracy: 0.7994 - val_loss: 0.5877 - learning_rate: 0.0010
Epoch 7/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 45ms/step - accuracy: 0.8615 - l

In [ ]:
with open("./modeldata.txt", mode="a", encoding="utf-8") as file:
    model.summary(print_fn=lambda x: file.write(f"{x}\n"))
    file.write(f"{str(history.history)}\n")


In [ ]:
model.save("model6.keras")